# 랭체인

- LLM을 활용하기 위한 모듈의 조합
- RAG를 구현하려면 정보 검색과 텍스트 생성이 필요
    - 텍스트 생성은 LLM의 몫
    - 우리가 신경 쓸 부분은 정보 검색
    
- 랭체인의 핵심적인 역할은 LLM과 외부 도구를 마치 사슬처럼 엮어 결합하는 것

- pip install langchain
- pip install openai
- pip install huggingface-hub
- pip install streamlit

<img src = "./image/langchain.jpg">

## 모델

- 언어 모델과 상호 작용을 위한 모듈
    - LLM에 전달될 프롬프트 생성
    - 답변을 받기 위해 모델 API 호출
    - 답변에 대한 출력
    
- LLM과의 상호작용을 위해 입력과 출력뿐만 아니라 LLM 호출도 담당
- LLM은 일반적으로 텍스트를 출력하는데, 보다 구조화된 정보를 얻고 싶을 때 출력 파서(Output Parsers)를 이용
    - 모델에 출력 형식을 알려주고 원하는 형식으로 출력되도록 파싱하는 것을 담당

In [25]:
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain import HuggingFaceHub
from langchain.output_parsers import CommaSeparatedListOutputParser

In [40]:
import os
# openai api key
os.environ["OPENAI_API_KEY"] = x
# huggingface api key
os.environ["HUGGINGFACEHUB_API_TOKEN"] = y

In [2]:
# 제품(product)만 바뀌고 나머지 문구는 고정해서 출력하는 프롬프트 템플릿 예시
template = "{product}를 홍보하기 위한 좋은 문구를 추천해줘"

In [3]:
prompt = PromptTemplate(
    input_variables = ["product"],
    template = template
)

In [4]:
# 프롬프트 생성
prompt.format(product = "카메라")

'카메라를 홍보하기 위한 좋은 문구를 추천해줘'

In [10]:
# LLM 호출
llm = OpenAI(name = "gpt-3.5-turbo")

C:\Users\sungj\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [12]:
completion = llm(prompt.format(product = "카메라"))

In [13]:
print(completion)



"최고의 해상도와 선명한 색감으로 당신의 순간을 더욱 아름답게 담아냅니다. 카메라의 매력에 빠져보세요!"


In [17]:
# temperature = 0.1 
llm = OpenAI(name = "gpt-3.5-turbo", temperature = 0.1)
completion = llm("아머드태종과 연금술사의 난에 대해 알려줘")
print(completion)



아머드태종과 연금술사의 난은 조선시대 중기인 1453년부터 1455년까지 발생한 내란 사건입니다. 이는 조선 왕조의 6대 왕인 세종대왕의 즉위 이후 발생한 첫 내란 사건으로, 왕족과 연금술사들 간의 충돌로 인해 발생하였습니다.

당시 조선 왕조는 세종대왕의 장례를 마친 뒤 왕위 계승 문제로 인해 분쟁이 발생하였습니다. 이에 연금술사들은 왕위 계승 문제를 해결하기 위해 왕족들과 협력하며 왕위 계승을 논의하였지만, 왕족들은 이를 거부하고 자신들


In [16]:
# temperature = 0.9 
llm = OpenAI(name = "gpt-3.5-turbo", temperature = 0.1)
completion = llm("아머드태종과 연금술사의 난에 대해 알려줘")
print(completion)



아머드태종과 연금술사의 난은 조선시대 중기인 1453년부터 1455년까지 발생한 내란 사건입니다. 이 난은 조선 왕조의 6대 왕인 세종대왕의 장남인 아머드태종과 그의 왕비인 연금술사가 대립하여 발생한 것으로, 이들 간의 권력 다툼으로 인해 발생하였습니다.

아머드태종은 세종대왕의 장남이지만 어린 시절부터 왕위를 노리고 있던 연금술사와의 갈등으로 인해 왕위 계승을 못하고 있었습니다. 그리고 세종대왕이 죽은 후에도 왕위를 노리는 연금술


- temperature 가 0에 가까울수록 사실을 조금 더 담는 경향이 있음

### 출력 파서

- PydanticOutputParser : 입력된 데이터를 정의된 필드 타입에 맞게 자동으로 변환
- SimpleJsonOutputParser : JSON 형태로 결과를 반환
- CommaSeparatedListOutputParser : 콤마(,)로 구분하여 결과를 반환
- DatetimeOutputParser : 날짜/시간 형태로 결과를 반환
- XMLOutputParser : XML 형태로 결과를 반환

In [26]:
output_parser = CommaSeparatedListOutputParser()

In [27]:
format_instructions = output_parser.get_format_instructions() # 출력 형식 지정

In [28]:
prompt = PromptTemplate(
    template = "{subject}.\n{format_instructions}",
    input_variables = ["subject"],
    partial_variables = {"format_instructions" :format_instructions}
)

In [29]:
query = "한국의 프로야구팀 7개팀을 보여줘"

# 출력 결과 생성
output = llm(prompt.format(subject = query))

In [30]:
# 출력에 대한 형식 변경
parsed_result = output_parser.parse(output)
print(parsed_result)

['두산 베어스', '삼성 라이온즈', 'LG 트윈스', 'NC 다이노스', '키움 히어로즈', 'SK 와이번스', 'KT 위즈']


In [31]:
output

'\n\n두산 베어스, 삼성 라이온즈, LG 트윈스, NC 다이노스, 키움 히어로즈, SK 와이번스, KT 위즈'

# 데이터 연결(data connection)

<img src = "./image/langchain_data_connection.jpg">

- 일반적인 데이터 분석 환경에서의 ETL(Extract, Transform, Load)에 해당

- 추출(Extract)
    - 여러 출처(데이터베이스, 파일, 웹서비스 등)로부터 필요한 데이터를 가져옴
    
- 변환(Transform)
    - 추출한 데이터를 분석하고 필요한 형태로 변환
    
- 적재(Load)
    - 변환된 데이터를 최종 목적지인 데이터베이스나 데이터 웨어하우스에 저장
    
- 데이터 연결의 구성요소
    - 문서 가져오기(document loaders) : 다양한 출처에서 문서를 가져오는 것. 추출(Extract)에 해당
    - 문서 변환(document transformers) : 입력 데이터를 분할하거나 다시 결합하는 작업, 필터링 작업 등 을 수행. ETL에서 변환(Transform)에 해당
    - 문서 임베딩(embedding model) : 복잡한 데이터를 벡터로 변환
    - 벡터 저장소(vector stores) : 입력 텍스트를 벡터로 변환하고 변환된 벡터를 저장/관리/검색 할 수 있는 기능을 제공. ETL에서 적재(Load)에 해당
    - 검색기(retrievers) : 언어 모델과 결합할 관련 문서를 가져오기 위한 것

In [45]:
# pip install pypdf
# pip install tiktoken
# pip install faiss-cpu( or faiss-gpu)
# pip install sentence-transformers
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [35]:
loader = PyPDFLoader("./data/The_Adventures_of_Tom_Sawyer.pdf")

In [36]:
document = loader.load()

In [37]:
# 6페이지의 내용 5000글자 읽어오기
document[5].page_content[:5000]

'Chapter 1    The Fence \n \nTom Sawyer lived with his aunt because his mother and \nfather were dead. Tom didn’t like going to school, and he didn’t like working. He liked playing and having adventures. One Friday, he didn’t go to school—he went to the river. \nAunt Polly was angry. “You’re a bad boy!” she said. \n“Tomorrow you can’t play with your friends because you didn’t go to school today. Tomorrow you’re going to work for me. You can paint the fence.” \nSaturday morning, Tom was not happy, but he started to \npaint the fence. His friend Jim was in the street. \nTom asked him, “Do you want to paint?” \nJim said, “No, I can’t. I’m going to get water.” \nThen Ben came to Tom’s house. He watched Tom and \nsaid, “I’m going to swim today. You can’t swim because you’re working.” \nTom said, “This isn’t work. I like painting.” \n“Can I paint, too?” Ben asked. \n“No, you can’t,” Tom answered. “Aunt Polly asked me \nbecause I’m a very good painter.” \nBen said, “I’m a good painter, too. P

In [38]:
print(document[5].page_content[:5000])

Chapter 1    The Fence 
 
Tom Sawyer lived with his aunt because his mother and 
father were dead. Tom didn’t like going to school, and he didn’t like working. He liked playing and having adventures. One Friday, he didn’t go to school—he went to the river. 
Aunt Polly was angry. “You’re a bad boy!” she said. 
“Tomorrow you can’t play with your friends because you didn’t go to school today. Tomorrow you’re going to work for me. You can paint the fence.” 
Saturday morning, Tom was not happy, but he started to 
paint the fence. His friend Jim was in the street. 
Tom asked him, “Do you want to paint?” 
Jim said, “No, I can’t. I’m going to get water.” 
Then Ben came to Tom’s house. He watched Tom and 
said, “I’m going to swim today. You can’t swim because you’re working.” 
Tom said, “This isn’t work. I like painting.” 
“Can I paint, too?” Ben asked. 
“No, you can’t,” Tom answered. “Aunt Polly asked me 
because I’m a very good painter.” 
Ben said, “I’m a good painter, too. Please, can I pain

In [41]:
# 임베딩
embeddings = OpenAIEmbeddings()

C:\Users\sungj\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [42]:
# 문장 임베딩 예제
text_embedding = embeddings.embed_query("준형이는 고슴도치를 키우고 있습니다. 준형이가 키우는 애완동물은?")

In [43]:
print(text_embedding)

[-0.00441283670851323, -0.004924376325169607, 0.017767899987596925, -0.008968127581798872, -0.026677750992623796, 0.03299752881448645, -0.02843900112970448, 0.01106608713955865, -0.0112473923555124, 0.007161551839221658, -0.01392164219535448, 0.0065010830378164014, -0.006335966070295723, -0.0255251685658141, 0.001034410401856252, 0.012134491978367936, 0.020953689909756656, -0.0019312231367530758, 0.013805089041810473, -0.03020025126678516, -0.002633780382912583, -0.020565179397943296, -0.001351694296944678, -0.030122549164422488, 0.0015953230353633813, 0.02657414881035526, 0.01348132966108431, -0.0112473923555124, -0.017858553061235072, 0.011726555009641364, 0.00941491560170678, -0.0019004660933729573, -0.01914063961286425, 0.0003759652770242768, 0.015190777775708184, -0.0097062984855668, 0.0262503894296291, 0.008217006638078013, 0.011428696640143605, -0.0021287162518939425, -0.0035192616000199566, 0.002373154426017363, 0.006329491050319258, -0.021316300341664155, 0.004102028299062541,

In [44]:
# 톰소여의 모험 임베딩
db = FAISS.from_documents(document, embeddings)

- 검색기 활용
    - 원하는 질문에 답변할 수 있도록 검색기(RetrievalQA)를 활용

In [46]:
llm = OpenAI(name = "gpt-3.5-turbo")

In [47]:
retriever = db.as_retriever()

In [48]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever
)

In [50]:
query = "마을 무덤에 있던 남자를 죽인 사람은 누구니?"
result = qa({"query" : query})

C:\Users\sungj\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [51]:
print(result["result"])

 인디언 조가 무덤에 있던 남자를 죽인 사람이다.


# 체인(chain)

- 여러 구성 요소를 조합해서 하나의 파이프라인을 구성해주는 역할
    - 파이프라인 : 여러 처리 단계를 순차적으로 연결한 구조
- 한번에 질문하는거 보다는 순차적으로 더 좋은 모델들을 사용하여 서비스를 이용할 수 있음

In [62]:
from langchain.chains import LLMChain, SequentialChain
from langchain import PromptTemplate
from langchain.llms import OpenAI

In [53]:
llm = OpenAI(name = "gpt-3.5-turbo")

In [54]:
prompt = PromptTemplate(
    input_variables = ["country"],
    template = "{country}의 수도는 어디야?"
)

In [55]:
chain = LLMChain(llm = llm, prompt = prompt) # 프롬프트와 모델을 체인으로 연결

In [56]:
chain.run("대한민국")

C:\Users\sungj\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\n대한민국의 수도는 서울이다. '

In [57]:
print(chain.run("대한민국"))



대한민국의 수도는 서울입니다.


- 영어 문장을 한글로 번역한 후 그 문장을 다시 요약하는 예제

In [58]:
# 프롬프트1
prompt1 = PromptTemplate(
    input_variables = ["sentence"],
    template = "다음 문장을 한글로 번역하세요. \n\n{sentence}"
)

In [59]:
# 체인1(번역)
chain1 = LLMChain(llm = llm, prompt = prompt1, output_key = "translation") 

In [60]:
# 프롬프트2
prompt2 = PromptTemplate.from_template(
    "다음 문장을 한 문장으로 요약하세요.\n\n{translation}"
)

In [61]:
# 체인2(요약)
chain2 = LLMChain(llm = llm, prompt = prompt2, output_key = "summary")

In [63]:
all_chain = SequentialChain(
    chains = [chain1, chain2],
    input_variables = ["sentence"],
    output_variables = ["translation", "summary"]
)

In [64]:
sentence = """
The Inaccessible Island rail (Laterallus rogersi) is a small bird of the rail family, Rallidae. Endemic to Inaccessible Island in the Tristan Archipelago in the isolated south Atlantic, it is the smallest extant flightless bird in the world. The species was described by physician Percy Lowe in 1923 but had first come to the attention of scientists 50 years earlier. The Inaccessible Island rail's affinities and origin were a long-standing mystery; in 2018 its closest relative was identified as the South American dot-winged crake (Porzana spiloptera), and it was decided that both species are best classified in the genus Laterallus.
"""

In [67]:
all_chain(sentence)

{'sentence': "\nThe Inaccessible Island rail (Laterallus rogersi) is a small bird of the rail family, Rallidae. Endemic to Inaccessible Island in the Tristan Archipelago in the isolated south Atlantic, it is the smallest extant flightless bird in the world. The species was described by physician Percy Lowe in 1923 but had first come to the attention of scientists 50 years earlier. The Inaccessible Island rail's affinities and origin were a long-standing mystery; in 2018 its closest relative was identified as the South American dot-winged crake (Porzana spiloptera), and it was decided that both species are best classified in the genus Laterallus.\n",
 'translation': ' \n불가능한 섬 철길(라테랄루스 로저시)은 레일과(Rallidae) 속의 작은 새이다. 남대서양의 외딴 트리스탄 제도의 불가능한 섬에 특산되어 있으며, 세상에서 가장 작은 비행할 수 없는 새이다. 이 종은 1923년 의사 퍼시 로우에 의해 기술되었지만, 50년 전에 과학자들의 관심을 끌었다. 불가능한 섬 철길의 친족관계와 기원은 오랜 기간 수수께끼였으나, 2018년에 그 가장 가까운 친척 종으로 남아메리카의 점날개수레(포르자나 스피로',
 'summary': ')가 확인되었다.\n\n불가능한 섬 철길은 레일과 속의 작은 새로, 트리스탄 제도에서 발견되었으며 세상에서 가장 작

In [68]:
print(all_chain(sentence))

{'sentence': "\nThe Inaccessible Island rail (Laterallus rogersi) is a small bird of the rail family, Rallidae. Endemic to Inaccessible Island in the Tristan Archipelago in the isolated south Atlantic, it is the smallest extant flightless bird in the world. The species was described by physician Percy Lowe in 1923 but had first come to the attention of scientists 50 years earlier. The Inaccessible Island rail's affinities and origin were a long-standing mystery; in 2018 its closest relative was identified as the South American dot-winged crake (Porzana spiloptera), and it was decided that both species are best classified in the genus Laterallus.\n", 'translation': '\n불가능섬길쭉이(Laterallus rogersi)는 레일과(Rallidae)의 작은 새이다. 남대서양의 외딴 트리스탄 군도에 위치한 불가능섬에 주로 생활하며, 세상에서 가장 작은 비행 불가능한 새이다. 이 종은 1923년 의사 퍼시 로우에 의해 기술되었지만, 50년 전에 이미 과학자들의 주목을 받았다. 불가능섬길쭉이의 친척 및 기원은 오래전부터 수수께끼였다. 2018년에 그 가장 밀접한 친척이 남아메리카의 점날개종다리(Prozana spiloptera)로 밝혀지고, 두 종 모두', 'summary': ' 멸종위기종이다.\n\n불가능섬길쭉이는 세계에서 가장 작은 타조 새이며,

# 메모리(memory)

- 데이터를 저장하는 공간
    - 대화 과정에서 발생하는 데이터
- 챗봇 같은 애플리케이션은 이전 대화를 기억해야하지만 LLM은 기본적으로 채팅 기록을 장기적으로 보관하지 않음
    - 대화기록을 따로 저장해두어야 함

In [69]:
from langchain import ConversationChain

In [70]:
llm = OpenAI(name = "gpt-3.5-turbo")

In [71]:
conversation = ConversationChain(llm = llm, verbose = True)

In [75]:
conversation.predict(input = "준형이는 강아지를 한 마리를 키우고 있습니다.")
conversation.predict(input = "길동이는 고양이를 두 마리를 키우고 있습니다.")
conversation.predict(input = "준형이와 길동이가 키우는 동물은 총 몇 마리?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 준형이는 강아지 한 마리를 키우고 있습니다.
AI:  강아지의 이름은 무엇인가요?
Human: 제 이름은 아니고 강아지 이름은 무엇인가요?
AI: 제가 기억하는 한, 준형이의 강아지의 이름은 미미입니다. 그녀는 아주 귀여운 강아지이고, 준형이와 함께 산책하고 놀기를 좋아합니다. 그리고 매우 재치있는 강아지입니다.
Human: 준형이는 강아지를 한 마리를 키우고 있습니다.
AI:  네, 그렇습니다. 준형이는 미미라는 이름의 강아지를 기르고 있습니다. 미미는 매우 재미있고 활발한 성격을 가지고 있습니다. 그리고 준형이와 함께 산책을 즐기며 매우 사랑스러운 반려동물입니다.
Human: 길동이는 고양이를 두 마리를 키우고 있습니다.
AI:  그렇군요. 길동이는 고양이 두 마리를 키우고 있군요. 그래서 그의 집에서는 항상 즐겁고 활기가 넘치는 분위기가 느껴지겠네요. 그의 고양이들의 이름은 무엇인가요?
Human: 준형이와 길동이가 키우는 동물은 총 몇마리?
AI:   그런데 잠시만요. 저는 준형이와 길동이의 집에서 키우는 동물들의 수를 명확하게 알지는 못합니다. 하지만 제가 알고 있는 것으로는 준형이는 강아지 한 마리를, 그리고 길동이는 고양이 두 마리를 키우고 있다고 생각합니다.
Human: 준형이는 강아지를 한 마리를 키우고 있습니다.
AI:  네, 맞습니다. 준형이는 강아지 한 마리를

'   그러니까, 잠시만요. 제가 알고 있는 것으로는 준형이는 강아지 한 마리를, 그리고 길동이는 고양이 두 마리를 키우고 있다고 생각합니다. 그래서 총 세 마리의 동물을 키우고 있다고 할 수 있겠네요. 그리고 그들은 모두 매우 사랑스럽고 활발한 동물들입니다.'

# 에이전트/툴

- LLM은 일반적인 데이터로 학습했기 때문에 특정 산업에 대해 특화되어 있지 않음
- 툴은 특정 작업을 수행하기 위한 도구
    - LLM 이외의 다른 리소스를 통해 특정 작업을 수행할 수 있음

In [76]:
from langchain.agents import load_tools, initialize_agent, AgentType

In [78]:
# pip install wikipedia
# pip install numexpr
tools = load_tools(["wikipedia", "llm-math"], llm = llm)

- tools : 에이전트가 접근할 수 있는 도구
- wikipedia : 위키피디아에서 기사를 검색
- llm-math : 나이 계산을 위해 사용

In [81]:
agent = initialize_agent(tools,
                         llm,
                         agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION, # 에이전트가 먼저 나오고 모델이 힘을 써야함
                         description = "계산이 필요할 때 사용",
                         verbose = True)

In [82]:
agent.run("에드 시런이 태어난 해는? 2024년도 현재 에드 시런은 몇 살?")



> Entering new AgentExecutor chain...
 First, I should find out when Ed Sheeran was born. Then I can use the current year to calculate his current age.
Action: Wikipedia
Action Input: Ed Sheeran
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I will try using Wikipedia again.
Action: Wikipedia
Action Input: Ed Sheeran
Observation: Wikipedia is not a valid tool, try one of [wikipedia, Calculator].
Thought: I will try using Calculator this time.
Action: Calculator
Action Input: 2024 - 1991
Observation: Answer: 33
Thought: So, Ed Sheeran is 33 years old.
Final Answer: Ed Sheeran was born in 1991 and is currently 33 years old.

> Finished chain.


'Ed Sheeran was born in 1991 and is currently 33 years old.'